Uses a local copy of the digital-land datasette to generate a csv file with the info needed to retire the duplicated/stale brownfield endpoints.

Download the digital land SQLite3 file from https://datasette.planning.data.gov.uk/digital-land. You can then set the path to it per the next section. Note how the paths of the files show how to do it from WSL2 to your windows host :-) 

In [ ]:
dbfile =r'/mnt/c/Users/MarkSmith/Downloads/digital-land.sqlite3'
csvfile = r'/mnt/c/Users/MarkSmith/Downloads/duplicates.csv'

In [ ]:
import sqlite3
import pandas as pd

# Create the connection
cnx = sqlite3.connect(dbfile)

cursor = cnx.cursor()

Create a view in your SQLite file that lists organisations and datasets where the dataset has more than a single source. I could have done this as an inner join but divide and conquer is how I prefer to work.

In [ ]:
cursor.execute("drop view duplicates")

duplicates_view_sql = """
create view duplicates as 
select org.name || '_' || ds.name as dupkey,  ds.name as dataset,  count( res.resource) as count 
from resource res
inner join resource_organisation ro on ro.resource = res.resource
inner join organisation org on org.organisation = ro.organisation
inner join resource_dataset rd on rd.resource = res.resource
inner join dataset ds on ds.dataset = rd.dataset
where res.end_date = ''
group by 1
having count > 1
"""

cursor.execute(duplicates_view_sql)

df = pd.read_sql_query("SELECT * FROM duplicates order by count desc", cnx)

df.head(15)

In [ ]:
bydataset = df.groupby('dataset')['count'].sum()
print(bydataset)
bydataset.plot.pie()

Now we can build out the endpoints that are candidates for retiring. Note how we use an aggregate of org name and dataset to identify each row. This assumes for each org, a dataset should be unique.

In [ ]:
sql = """
select org.name || '_' || ds.name as thiskey, org.name as organization, ds.name as dataset, 
res.start_date as res_start_date, res.resource, ep.endpoint, ep.endpoint_url, ep.start_date as ep_start_date, ep.end_date

from resource res
inner join resource_organisation ro on ro.resource = res.resource
inner join organisation org on org.organisation = ro.organisation
inner join resource_dataset rd on rd.resource = res.resource
inner join dataset ds on ds.dataset = rd.dataset
inner join resource_endpoint re on re.resource = res.resource
inner join endpoint ep on ep.endpoint = re.endpoint

where res.end_date = ''
and thiskey in (select dupkey from duplicates)
order by 2,3,4, ep.start_date 
"""

df = pd.read_sql_query(sql, cnx)

df.to_csv(csvfile)

df.head()

In [ ]:
sql = """
select * from endpoint ep
where ep.start_date = '' and ep.end_date = ''
"""

df = pd.read_sql_query(sql, cnx)

df.head()